In [1]:
import requests
import pandas as pd
from xp_prediction.prep_data import FIELDS_TO_WEIGHT, calculate_weighted_field, get_team_and_fixture_info_current_season
import numpy as np

In [2]:
bootstrap_static = "https://fantasy.premierleague.com/api/bootstrap-static/"
response = requests.get(bootstrap_static)
data = response.json()
all_player_ids = [data['elements'][i]['id'] for i in range(len(data['elements']))]
next_gw = 10
players_raw_df = pd.DataFrame(data["elements"])
teams_df = pd.DataFrame(data["teams"])


# for i in all_player_ids:
#     player_summary = f"https://fantasy.premierleague.com/api/element-summary/{i}/"
#     response = requests.get(player_summary)
#     player_data = response.json()
#     player_df = pd.DataFrame(player_data['history'])





In [3]:
player_summary = f"https://fantasy.premierleague.com/api/element-summary/{16}/"
response = requests.get(player_summary)
player_data = response.json()
player_df = pd.DataFrame(player_data['history'])
player_fixtures_df = pd.DataFrame(player_data['fixtures'])

In [4]:
player_df.head()

,element,fixture,opponent_team,total_points,was_home,kickoff_time,team_h_score,team_a_score,round,modified,...,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,value,transfers_balance,selected,transfers_in,transfers_out
0,16,9,14,3,False,2025-08-17T15:30:00Z,0,1,1,False,...,1,0.15,0.09,0.24,1.52,100,0,1602493,0,0
1,16,11,11,6,True,2025-08-23T16:30:00Z,5,0,2,False,...,1,0.11,0.03,0.14,0.17,100,43708,1870918,152171,108463
2,16,25,12,0,False,2025-08-31T15:30:00Z,1,0,3,False,...,0,0.00,0.00,0.00,0.00,99,-986673,783017,55049,1041722
3,16,31,16,0,True,2025-09-13T11:30:00Z,3,0,4,False,...,0,0.00,0.00,0.00,0.00,98,-368029,500371,5237,373266
4,16,41,13,1,True,2025-09-21T15:30:00Z,1,1,5,False,...,0,0.04,0.21,0.25,0.34,98,-95794,415767,1687,97481


In [5]:
data = {}
for key in FIELDS_TO_WEIGHT:
    data[key] = calculate_weighted_field(player_df, key, 10)

In [6]:
data

{'assists': 0.0,
 'goals_scored': 0.2,
 'minutes': 75.86666666666666,
 'goals_conceded': 0.13333333333333333,
 'influence': 17.133333333333333,
 'creativity': 30.44,
 'threat': 23.4,
 'ict_index': 7.086666666666668,
 'bonus': 0.5333333333333333,
 'yellow_cards': 0.0,
 'saves': 0.0,
 'expected_assists': 0.3873333333333333,
 'expected_goal_involvements': 0.594,
 'expected_goals': 0.20666666666666667,
 'expected_goals_conceded': 0.3353333333333333}

In [7]:
get_team_and_fixture_info_current_season(11, teams_df, player_fixtures_df)

[{'opposition_strength': 2,
  'opponent_strength_attack': 1050,
  'opponent_strength_defense': 1050,
  'was_home': 0}]

In [8]:
from api_data_fetcher import FPLDataFetcher

In [9]:
fetcher = FPLDataFetcher()
player_data = fetcher.featurize_all_players(as_type=list)

Featurising players: 100%|██████████| 746/746 [00:43<00:00, 17.00it/s]


In [10]:
pd.DataFrame(player_data).head()    

,assists,goals_scored,minutes,goals_conceded,influence,creativity,threat,ict_index,bonus,yellow_cards,...,expected_goals_conceded,opposition_strength,opponent_strength_attack,opponent_strength_defense,was_home,goalkeeper,defender,midfielder,forward,pid
0,0.000000,0.000000,90.0,0.2,10.40,0.033333,0.0,1.046667,0.0,0.0,...,0.512667,2,1050,1050,0,1,0,0,0,1
1,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,2,1050,1050,0,1,0,0,0,2
2,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,2,1050,1050,0,1,0,0,0,3
3,0.000000,0.000000,0.0,0.0,0.00,0.000000,0.0,0.000000,0.0,0.0,...,0.000000,2,1050,1050,0,1,0,0,0,4
4,0.266667,0.133333,90.0,0.2,34.44,7.713333,11.8,5.386667,2.4,0.0,...,0.512667,2,1050,1050,0,0,1,0,0,5


In [11]:
from xp_prediction import XPPredictor

In [12]:
predictor = XPPredictor('models/20251028-111058-6581b73f')

/Users/iwanpavord/Desktop/auto-fpl/graph_traversal_bot/xp_prediction/xp_predictor.py:21: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(model_path, map_loc

In [19]:
from bot import GraphTraversalBot

ImportError: attempted relative import with no known parent package